In [3]:
#Importing libraries 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [16]:
filename_data = os.path.join(os.getcwd(),"all_data.csv")
filename_orders = os.path.join(os.getcwd(),"all_orders.csv")

all_data = pd.read_csv(filename_data)
all_orders = pd.read_csv(filename_orders)

### What are the top selling products?

In [21]:
cancelled_orders = all_orders[all_orders['status_item'] == 'Cancelled'].copy()
returned_orders = all_orders[all_orders['status_item'] == 'Returned'].copy()
complete_orders = all_orders[all_orders['status_item'].isin(['Complete', 'Shipped'])].copy()
complete_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97626 entries, 1 to 177353
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_orderproducts         97626 non-null  int64  
 1   order_id_all             97626 non-null  int64  
 2   user_id_all              97626 non-null  int64  
 3   product_id_all           97626 non-null  int64  
 4   inventory_item_id_all    97626 non-null  int64  
 5   status_item              97626 non-null  object 
 6   created_at_item          97626 non-null  object 
 7   shipped_at_item          97626 non-null  object 
 8   delivered_at_item        44502 non-null  object 
 9   returned_at_item         0 non-null      object 
 10  sale_price_all           97626 non-null  float64
 11  status_order             97626 non-null  object 
 12  gender                   97626 non-null  object 
 13  created_at_order         97626 non-null  object 
 14  returned_at_order        0

In [24]:
top_selling_products = complete_orders.groupby(['product_id_all', 'name'])['num_of_item'].sum().reset_index().sort_values('num_of_item', ascending=False)
top_selling_products.head(10)

,product_id_all,name,num_of_item
17973,18795,Life is Good Men's King Of The Grill Short Sle...,30
16252,17045,Bayside Apparel Adult USA-Made Long-Sleeve Poc...,29
7933,8344,Port AuthorityÂ® - Ladies CRBNÂ® Trench,29
23575,24560,Wigwam Men's King Cotton Crew Length Crew Sock,27
23716,24705,San Francisco 49ers NFL Mens Sherpa Slide Slip...,27
26007,27059,ESPN Men's Microfleece Lounge Pants,26
8203,8624,United Face Womens New Leather Moto Jacket,26
24036,25030,Kentwool Men's Tour Standard Sock,26
19583,20450,7 For All Mankind Men's Ombre Plaid Blazer,26
26932,28012,Free Country Men's Island Palm Plaid Free C Bo...,26


### Summarizing Order Data

In [ ]:
us_events = 